In [1]:
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func, select, Table, MetaData
import matplotlib as plt

In [2]:
#read graduation csv
df = pd.read_csv("graduation_data.csv")

#check graduation data
df.head(10)

,Board Number,Year,Board Type,Region,Grade 10 OSSLT Results,Progress in Grade 10 OSSLT Results,Credit Accumulation by the end of Grade 10,Progress in Credit Accumulation by the end of Grade 10,Credit Accumulation by the end of Grade 11,Progress in Credit Accumulation by the end of Grade 11,Four Year Graduation Rate 2017-2018 Grade 9 Cohort,Progress in Four Year Graduation Rate 2017-2018 Grade 9 Cohort
0,B28010,2021-2022,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.718,0.018
1,B67202,2021-2022,Roman Catholic,East Region,0.86,0.0,0.84,0.00,0.87,-0.03,0.880,0.004
2,B66010,2021-2022,Public,West Region,0.81,0.0,0.81,0.03,0.78,-0.01,0.810,0.066
3,B66001,2021-2022,Public,West Region,0.75,0.0,0.67,-0.01,0.71,-0.01,0.711,0.011
4,B67164,2021-2022,Roman Catholic,West Region,0.85,0.0,0.73,-0.08,0.81,-0.01,0.797,-0.038
5,B67008,2021-2022,Roman Catholic,West Region,0.82,0.0,0.82,0.01,0.85,0.05,0.868,0.027
6,B67172,2021-2022,Roman Catholic,East Region,0.84,0.0,0.80,-0.09,0.84,-0.06,0.882,0.014
7,B66311,2021-2022,Public,East Region,0.93,0.0,0.78,-0.01,0.84,-0.06,0.915,0.008
8,B67300,2021-2022,Roman Catholic,West Region,0.85,0.0,0.91,-0.04,0.92,-0.05,0.960,0.062
9,B67318,2021-2022,Roman Catholic,Toronto Region,0.95,0.0,0.87,0.02,0.91,-0.02,0.938,0.021


In [3]:
#splitting the year variable to a single year
df["Year"] = df["Year"].str.split("-").str[0]

In [4]:
#changing the year variable to an integer
df["Year"] = df["Year"].astype(int)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 12 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Board Number                                                    144 non-null    object 
 1   Year                                                            144 non-null    int32  
 2   Board Type                                                      144 non-null    object 
 3   Region                                                          144 non-null    object 
 4   Grade 10 OSSLT Results                                          144 non-null    float64
 5   Progress in Grade 10 OSSLT Results                              144 non-null    float64
 6   Credit Accumulation by the end of Grade 10                      144 non-null    float64
 7   Progress in Credit Accumulation by the end of Grade 1

In [6]:
df.sample(10)

,Board Number,Year,Board Type,Region,Grade 10 OSSLT Results,Progress in Grade 10 OSSLT Results,Credit Accumulation by the end of Grade 10,Progress in Credit Accumulation by the end of Grade 10,Credit Accumulation by the end of Grade 11,Progress in Credit Accumulation by the end of Grade 11,Four Year Graduation Rate 2017-2018 Grade 9 Cohort,Progress in Four Year Graduation Rate 2017-2018 Grade 9 Cohort
91,B66150,2022,Public,West Region,0.86,0.0,0.76,-0.01,0.78,-0.03,0.803,-0.020
11,B28100,2021,Public,North Region,0.72,0.0,0.73,-0.04,0.82,0.06,0.814,-0.068
26,B67113,2021,Roman Catholic,Central Region,0.90,0.0,0.81,-0.07,0.87,-0.06,0.914,0.010
63,B66087,2021,Public,Central Region,0.84,0.0,0.75,-0.02,0.78,-0.03,0.818,0.007
83,B28100,2022,Public,North Region,0.72,0.0,0.73,-0.04,0.82,0.06,0.867,0.010
32,B29025,2021,Roman Catholic,North Region,0.80,0.0,0.73,-0.12,0.76,-0.09,0.828,0.021
0,B28010,2021,Public,North Region,0.72,0.0,0.69,-0.02,0.71,-0.08,0.718,0.018
117,B67180,2022,Roman Catholic,East Region,0.88,0.0,0.83,-0.01,0.89,-0.02,0.911,0.013
110,B66206,2022,Public,East Region,0.79,0.0,0.75,-0.05,0.74,-0.07,0.827,0.013
67,B66176,2021,Public,West Region,0.85,0.0,0.82,0.00,0.81,0.00,0.778,0.061


In [25]:
X = df[[col for col in df.columns if col != 'Region' and col !='Board Type' and col != "Board Number" and col != "Four Year Graduation Rate 2017-2018 Grade 9 Cohort"]]

In [26]:
X.shape

(144, 9)

In [27]:
y = df['Four Year Graduation Rate 2017-2018 Grade 9 Cohort']

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [29]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)


ValueError: could not convert string to float: 'East Region'

In [31]:
x_test_pred = model.predict(X_test)


ValueError: could not convert string to float: 'East Region'

In [33]:
#results = pd.DataFrame({"Prediction": x_test_pred, "Actual": y_test}).head()
#results

In [ ]:
# Saving the model for later use in the app
with open("enrollment_prediction_model.pkl", "wb") as file:
  pickle.dump(model, file)